In [ ]:
#import statements
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Draw
import sklearn
import matplotlib.pyplot as plt

# Loading raw data

This data has been gathered from https://pubs.acs.org/doi/suppl/10.1021/ci9901338/suppl_file/ci9901338_s.pdf, with the SMILEs already attached.

I have already edited some of the files to make it cleaner to load. This involved the following:
    1. Replacing any tabs with spaces, replacing double spaces with a single space.
    2. Combining train.smi and test1.smi into a single dataset.
    3. Keeping only the CAS_Number, Name, Log S and Smiles columns.
    4. Saving the data as: alldata.csv and test2.csv
    
If you wish, you may do this for yourself, by downloading the raw files from here: http://cheminformatics.org/datasets/huuskonen/index.html

Otherwise, you are welcome to use the .csv files generated for you.


In [ ]:
data = pd.read_csv("../data/alldata.csv", index_col=0)
data.head()

In [ ]:
#count the number of data entries
print(len(data))

In the paper, they have 1297 compounds, whilst we have 1291 here. 

Why is this?

In [ ]:
#think, then double click on the markdown cell below.


<font color="white">
<i> It is likely that we have less due to the SMILEs strings - if they were not able to be parsed by the data owner, they were excluded from the dataset.</i></font>

Let's do a check on a molecule to see if we have accurate data.

We can use pandas directly to do this:

    data[data["Column"] == "String"]] 
    
Replace 'Column' and 'String' with a molecule of your choice and see if you can find it in the raw data [table](https://pubs.acs.org/doi/suppl/10.1021/ci9901338/suppl_file/ci9901338_s.pdf).

In [ ]:
data[data["Name"]=="acetamide"]

We will be working with Smiles strings as input to our modelling work, so we need to make sure these are unique.
 
You can use value_counts() on a column to find the number of matches to each unique value.
e.g.

    data["Name"].value_counts()
    
The output is generated so that values with the highest counts are listed first.
    
Do this for the Smiles column.

In [ ]:
#work here


We can specify to only look at those that have a count > 1.
Don't forget to save your value_counts() as a variable first.

    counts[counts > 1]


In [ ]:
#work here

Let's look at the top example.

In [ ]:
data[data["Smiles"]==None] # Replace None with the relevant smiles

Both endrin and dieldrin have the same chemical formula: C12H8Cl6O

We can draw the molecules to see how RDKit visualises the SMILES representation - We'll look more at this in another notebook.

To do this we need to do a few steps first:
    1. Chem.MolFromSmiles(smiles)
    2. Draw.MolToImage(mol)

Take a look at the rdkit documentation for more information on these functions http://www.rdkit.org/Python_Docs/rdkit.Chem.rdmolfiles-module.html
    
Step 1 takes the smiles string as an argument, and converts it into an RDKit molecule type, so that we can perform further processing using rdkit features.
Step 2 takes the mol as an argument, and constructs an image.

In [ ]:
smiles = "ClC4=C(Cl)C5(Cl)C3C1CC(C2OC12)C3C4(Cl)C5(Cl)Cl"
mol = Chem.MolFromSmiles(smiles)

Draw.MolToImage(mol)

An issue with canonical smiles strings is that tautomers of the same molecule will typically be represented by the same SMILES
string. It is therefore often better to use Inchis which can be non-standardised, and are able to represent tautomers.

However, as we have no other tautomer information, we will stick with SMILEs.

This means that if this is used as input, even though for different tautomers, the descriptors generated will be the same.
We will have repeated data, which can cause a problem if one version of the repeat is in the test, and another in the training! 

If endrin, with a Log S of -6.18 is present in the training set, and dieldrin is present in the test set with a Log S of -6.29, we can always expect dieldrin to have **at least** an error of -0.11 (not considering training error).

What should you do? 
1. Keep all?
2. Remove all?
3. Keep one?
    - Which?

My preference here would be to try to find a way to represent these molecules so that they can become different entities, and then to make sure they end up in the same training/test set. 
However, due to limitations of the data, we'll just remove them.

How would you save the indexes of this 'counts' dataframe as list?
Remember to only save those that have a value_counts() > 1!

In [ ]:
multiple_mols = counts[counts > 1].index.tolist()
print(multiple_mols)

We can do a quick one liner for excluding all of these from the dataframe.

You've already seen:

       data[data["Name"] == "X"]
       
We can check for presence in a list according to:

        data[data["Name"].isin(X)]
            
Where X is your list variable.

How would you change this line, so it becomes 

    '"Name" is not in list X'?
    
Hint:<br><font size=5>~</font>

In [ ]:
unique_data = data[None] #Replace None with the code.

#We then need to reset the index, so we can avoid copy errors
unique_data.reset_index(drop=True,inplace=True)
unique_data.head()

We finally need to check that all our data has a Log S value, as this is what we are trying to predict.

In [ ]:
assert(np.all(unique_data["Log S"].isnull())==False)

In the 'data sets' paragraph:
    
<blockquote>A set of 1297 organic compounds was extracted from these databases and was divided into a training set of 884 compounds and a randomly chosen test set of 413 compounds.</blockquote>

You can use sklearn's train_test_split function for this! [Wiki here](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) 

Have a look at the wiki, and try and split the unique_data into training, and 413 'test' molecules. Don't forget, due to SMILES, we have slightly fewer!

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(unique_data, test_size=413) #replace None with the correct code

#save these as the correct csv files.
train.to_csv("../train.csv")
test.to_csv("../test1.csv")

print(len(train))
print(len(test))

We can then use a handy pandas function to get some statistics.

    data.describe()

    data["Name"].describe()

In [ ]:
#work here 

How does our means, minimums, maximums and standard deviations compare to those in the paper?

<blockquote>The Log S values of the training set ranged from -11.62 to +1.58 with a mean of -2.70 and standard deviation of 2.01. For the testing set, the smallest log S value was -10.41 and the largest +1.13. The mean and standard deviation were -2.77 and 2.07, respectively. </blockquote>

We could compare our train and test data in a box plot.

In [ ]:
# we need to combine into a list so we can plot them
data = [train["Log S"],test["Log S"]]

In [ ]:
fig = plt.figure(1, figsize=(9, 6))

# Create an axes instance
ax = fig.add_subplot(111)
ax.boxplot(data)
ax.set_ylabel("Log S")
ax.set_xticklabels(["Train","Test"])
plt.show()

Does the training data look to be representative of the test data?


## What about the second test set?

Do the plot again, but also with the Log S data from test

In [ ]:
test2 = pd.read_csv("../data/test2.csv", index_col=0)
test2.head()

In [ ]:
# work here